# W4_Preventing Overfitting in Decision Trees

- stopping condition 1: having no mistakes
- stopping condition 2: havin no more features to split on
- early stopping condition 1: reaching maximum depth
- early stopping condition 2: reaching minimum node size
- early stopping condition 3: not reaching minimum gain in error reduction

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('lending-club-data.csv')
data['safe_loans'] = data['bad_loans'].apply(lambda x: 1 if x==0 else -1)
features = ['grade', 'term', 'home_ownership', 'emp_length']
target = 'safe_loans'
data = pd.get_dummies(data[features+[target]])
data.head()

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_n/a
0,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [3]:
train_index = list(pd.read_json('module-6-assignment-train-idx.json')[0])
validation_index = list(pd.read_json('module-6-assignment-validation-idx.json')[0])
train = data.iloc[train_index]
validation = data.iloc[validation_index]

In [4]:
# used to decide the best splitting feature
def count_mistakes(labels_in_node):
    # corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    safe = (labels_in_node==1).sum()
    risky = (labels_in_node==-1).sum()
    if safe >= risky:
        return risky
    else:
        return safe

In [5]:
def best_splitting_feature(data, features, target):
    best_feature = None
    best_error = float('inf')
    
    for feature in features:
        left_split = data[data[feature]==0]
        right_split = data[data[feature]==1]
        left_error = count_mistakes(np.array(left_split[target]))
        right_error = count_mistakes(np.array(right_split[target]))
        total_error = left_error + right_error
        if total_error < best_error:
            best_feature = feature
            best_error = total_error
            
    return best_feature

In [6]:
def create_leaf(target_values):
        leaf = {'splitting_feature' : None,
                'left' : None,
                'right' : None,
                'is_leaf': True}
        safe = (target_values==1).sum()
        risky = (target_values==-1).sum()
        if safe >= risky:
            leaf['prediction'] = 1
        else:
            leaf['prediction'] = -1
        return leaf

In [7]:
def create_decision_tree(data, features, target, current_depth=0, max_depth=10, min_node_size=1, min_error_reduction=0.0):
    remaining_features = features[:]
    target_values = data[target]
    
    print('--------------------------------------------------------------------')
    print('Subtree, current depth:{}, data point:{}'.format(current_depth, len(data)))
    
    # stopping condition 1
    if count_mistakes(target_values) == 0:
        print('Stopping condition 1 reached. All data points have the same target value.')
        return create_leaf(target_values)
    
    # stopping condition 2
    if len(remaining_features) == 0:
        print('Stopping condition 2 reached. No remaining features.')
        return create_leaf(target_values)
    
    # early stopping condition 1
    if current_depth == max_depth:
        print('Early stopping condition 1 reached. Reached maximum depth.')
        return create_leaf(target_values)
    
    # early stopping condition 2
    if len(data) <= min_node_size:
        print('Early stopping condition 2 reached. Reached minimum node size.')
        return create_leaf(target_values)
    
    splitting_feature = best_splitting_feature(data, remaining_features, target)
    
    left_split = data[data[splitting_feature]==0]
    right_split = data[data[splitting_feature]==1]
    
    # early stopping condition 3
    error_before = count_mistakes(target_values) / float(len(data))
    error_left = count_mistakes(left_split[target])
    error_right = count_mistakes(right_split[target])
    error_after = (error_left + error_right) / float(len(data))
    if (error_before - error_after) <= min_error_reduction:
        print('Early stopping condition 3 reached. Minimum error reduction.')
        return create_leaf(target_values)      
    
    remaining_features.remove(splitting_feature)
    
    if (len(left_split) == len(data) or len(right_split) == len(data)):
        print('All feature values are the same. Create leaf node')
        return create_leaf(target_values)
    
    left_tree = create_decision_tree(left_split, remaining_features, target, current_depth + 1, max_depth, min_node_size, min_error_reduction)  
    right_tree = create_decision_tree(right_split, remaining_features, target, current_depth + 1, max_depth, min_node_size, min_error_reduction)  
        
    return {'is_leaf': False, 
            'prediction': None,
            'splitting_feature': splitting_feature,
            'left': left_tree, 
            'right': right_tree}

In [8]:
features = list(data.columns)
features.remove('safe_loans')
target = 'safe_loans'
simple_dicision_tree = create_decision_tree(train, features, target, max_depth=6, min_node_size=100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, current depth:0, data point:37224
--------------------------------------------------------------------
Subtree, current depth:1, data point:9223
--------------------------------------------------------------------
Subtree, current depth:2, data point:9122
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, current depth:2, data point:101
--------------------------------------------------------------------
Subtree, current depth:3, data point:96
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, current depth:3, data point:5
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, current depth:1, data point:28001
----------------------------------------

In [9]:
decision_tree = create_decision_tree(train, features, target, max_depth=6, min_node_size=0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, current depth:0, data point:37224
--------------------------------------------------------------------
Subtree, current depth:1, data point:9223
--------------------------------------------------------------------
Subtree, current depth:2, data point:9122
--------------------------------------------------------------------
Subtree, current depth:3, data point:8074
--------------------------------------------------------------------
Subtree, current depth:4, data point:5884
--------------------------------------------------------------------
Subtree, current depth:5, data point:3826
--------------------------------------------------------------------
Subtree, current depth:6, data point:1693
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, current depth:6, data point:2133
Early stopping condition 1 reached. Reached maximum 

In [10]:
def make_predicion(tree, x, annotate=False):
    if tree['is_leaf']:
        if annotate:
            print('reach leaf!')
        return tree['prediction']
    else:
        splitting_feature_value = x[tree['splitting_feature']]
        if annotate:
            print('splitting feature:', tree['splitting_feature'])
            print('splitting feature value:', splitting_feature_value)
        if splitting_feature_value == 0:
            return make_predicion(tree['left'], x, annotate)
        else:
            return make_predicion(tree['right'], x, annotate)

In [11]:
x = validation.iloc[0]
# 1st tree
make_predicion(simple_dicision_tree, x, annotate=True)

splitting feature: term_ 36 months
splitting feature value: 0
splitting feature: grade_A
splitting feature value: 0
reach leaf!


-1

In [12]:
# 2nd tree
make_predicion(decision_tree, x, annotate=True)

splitting feature: term_ 36 months
splitting feature value: 0
splitting feature: grade_A
splitting feature value: 0
splitting feature: grade_B
splitting feature value: 0
splitting feature: grade_C
splitting feature value: 0
splitting feature: grade_D
splitting feature value: 1
reach leaf!


-1

In [13]:
def compute_classification_error(tree, data):
    predictions_list = []
    for i in range(len(data)):
        predictions_list.append(make_predicion(tree, data.iloc[i]))
    predictions = np.array(predictions_list)
    error_num = (predictions!=data['safe_loans']).sum()
    error_ratio = error_num / len(data)
    return error_ratio

In [14]:
compute_classification_error(simple_dicision_tree, validation)

0.38367083153813014

In [15]:
compute_classification_error(decision_tree, validation)

0.38377854373115039

In [16]:
# explore the effect of max_depth
model_1 = create_decision_tree(train, features, target, max_depth=2, min_node_size=1, min_error_reduction=-1)
model_2 = create_decision_tree(train, features, target, max_depth=6, min_node_size=1, min_error_reduction=-1)
model_3 = create_decision_tree(train, features, target, max_depth=14, min_node_size=1, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, current depth:0, data point:37224
--------------------------------------------------------------------
Subtree, current depth:1, data point:9223
--------------------------------------------------------------------
Subtree, current depth:2, data point:9122
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, current depth:2, data point:101
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, current depth:1, data point:28001
--------------------------------------------------------------------
Subtree, current depth:2, data point:23300
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, current depth:2, data point:4701
Early stopping condition 1 reached. Reached 

All feature values are the same. Create leaf node
--------------------------------------------------------------------
Subtree, current depth:5, data point:45
All feature values are the same. Create leaf node
--------------------------------------------------------------------
Subtree, current depth:2, data point:101
--------------------------------------------------------------------
Subtree, current depth:3, data point:96
--------------------------------------------------------------------
Subtree, current depth:4, data point:85
All feature values are the same. Create leaf node
--------------------------------------------------------------------
Subtree, current depth:4, data point:11
All feature values are the same. Create leaf node
--------------------------------------------------------------------
Subtree, current depth:3, data point:5
All feature values are the same. Create leaf node
--------------------------------------------------------------------
Subtree, current depth:1, d

All feature values are the same. Create leaf node
--------------------------------------------------------------------
Subtree, current depth:2, data point:4701
All feature values are the same. Create leaf node


In [17]:
for model in [model_1, model_2, model_3]:
    print(round(compute_classification_error(model, validation), 3))

0.398
0.384
0.377


In [18]:
for model in [model_1, model_2, model_3]:
    print(round(compute_classification_error(model, train), 3))

0.4
0.382
0.376


In [19]:
# measure the complexity of the tree
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [20]:
for model in [model_1, model_2, model_3]:
    print(count_leaves(model))

4
19
41


In [21]:
# Explore the effect of min_error
model_4 = create_decision_tree(train, features, target, max_depth=6, min_node_size=0, min_error_reduction=-1)
model_5 = create_decision_tree(train, features, target, max_depth=6, min_node_size=0, min_error_reduction=0.0)
model_6 = create_decision_tree(train, features, target, max_depth=6, min_node_size=0, min_error_reduction=5)

--------------------------------------------------------------------
Subtree, current depth:0, data point:37224
--------------------------------------------------------------------
Subtree, current depth:1, data point:9223
--------------------------------------------------------------------
Subtree, current depth:2, data point:9122
--------------------------------------------------------------------
Subtree, current depth:3, data point:8074
--------------------------------------------------------------------
Subtree, current depth:4, data point:5884
--------------------------------------------------------------------
Subtree, current depth:5, data point:3826
--------------------------------------------------------------------
Subtree, current depth:6, data point:1693
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, current depth:6, data point:2133
Early stopping condition 1 reached. Reached maximum 

Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, current depth:0, data point:37224
Early stopping condition 3 reached. Minimum error reduction.


In [22]:
for model in [model_4, model_5, model_6]:
    print(round(compute_classification_error(model, validation), 3))

0.384
0.384
0.503


In [23]:
for model in [model_4, model_5, model_6]:
    print(count_leaves(model))

19
13
1


In [24]:
# Explore the effect of min_node_size
model_7 = create_decision_tree(train, features, target, max_depth=6, min_node_size=0, min_error_reduction=-1)
model_8 = create_decision_tree(train, features, target, max_depth=6, min_node_size=2000, min_error_reduction=-1)
model_9 = create_decision_tree(train, features, target, max_depth=6, min_node_size=50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, current depth:0, data point:37224
--------------------------------------------------------------------
Subtree, current depth:1, data point:9223
--------------------------------------------------------------------
Subtree, current depth:2, data point:9122
--------------------------------------------------------------------
Subtree, current depth:3, data point:8074
--------------------------------------------------------------------
Subtree, current depth:4, data point:5884
--------------------------------------------------------------------
Subtree, current depth:5, data point:3826
--------------------------------------------------------------------
Subtree, current depth:6, data point:1693
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, current depth:6, data point:2133
Early stopping condition 1 reached. Reached maximum 

In [25]:
for model in [model_7, model_8, model_9]:
    print(round(compute_classification_error(model, validation), 3))

0.384
0.385
0.503


In [26]:
for model in [model_7, model_8, model_9]:
    print(count_leaves(model))

19
12
1
